In [1]:
from video_tools import load_video, save_video
import numpy as np
import spams
from scipy.sparse import csc_matrix

In [4]:
def Omega(M, video_size=(100,100)):
    """
    Valor de la norma Omega de la matriz D. los valores de video_size se usan
    para calcular los indices g que necesita la norma
    """
    video = M.reshape((video_size[0], video_size[1], M.shape[1]))
    
    z = np.zeros(M.shape[1])
    for i in range(1, video_size[0] - 1): 
        for j in range(1, video_size[1] - 1): 
            window = video[i-1:i+2, j-1:j+2, :]
            vec_window = window.reshape(9, window.shape[2])
            z+= np.max(vec_window, axis=0)
    
    omega = np.sum(z)
    
    return omega

def get_groups(video_size):
    num_elements = video_size[0] * video_size[1]
    num_groups = (video_size[0] - 2) * (video_size[1] - 2)
    
    grid = np.arange(num_elements).reshape(video_size)
    
    group_count = 0
    
    rows = []
    columns = []
    
    for i in range(1, video_size[0] - 1): 
        for j in range(1, video_size[1] - 1): 
            window = grid[i-1:i+2, j-1:j+2]
            
            columns.extend([group_count] * 9)
            rows.extend(window.flatten())
                
            group_count += 1
    
    out = csc_matrix((np.ones(len(rows)), (rows, columns)), 
                     shape=(num_elements, num_groups),
                     dtype=np.bool)
    
    return out

def augmented_lagrangian(L, S, Y, mu, lamb, D):
    nuc = np.linalg.norm(L, ord='nuc')
    omega = Omega(S)
    tmp = D - L - S
    frob = np.linalg.norm(tmp, ord='fro')
    return nuc + lamb * omega + np.sum(Y * tmp) + 0.5 * mu * frob ** 2

def J(Y, lamb):
    #if not scipy.sparse.issparse(Y):
    #    Y = scipy.sparse.csc_matrix(Y)
    Y_norm = np.linalg.norm(Y)
    return max(Y_norm, 1/lamb * Y_norm)


def soft_threshold(X, eps):
    out = np.zeros(X.shape)
    out[X > eps] = X[X > eps] - eps
    out[X < -eps] = X[X < -eps] + eps
    return out


def video_segmentation(D, video_size, max_iter=500, mu_0=0.001, lamb=0.01):
    Y_k = D / J(D, lamb)
    S_k = np.zeros(D.shape)
    #S_k = D - Y_k
    
    print('Getting groups')
    groups = get_groups(video_size)
    
    print('Building tree')
    tree = {'eta_g': np.ones(groups.shape[1]), 
            'groups': csc_matrix((groups.shape[1], groups.shape[1]), dtype=np.bool), 
            'groups_var': groups}
    
    err = [] 
    for i in range(1, max_iter):
        mu_k = mu_0 * i  # mu_i crece de forma lineal

        print("Iteracion {}".format(i))
        # Resolver L_{k+1} = argmin_L L(L, S_k, Y_k, mu_k)
        tmp = D - S_k + (1 / mu_k) * Y_k
        #print('Converting to sparse')
        #tmp = scipy.sparse.csc_matrix(tmp)
        
        print("Starting SVD")
        U, S, V = np.linalg.svd(D - S_k + (1 / mu_k) * Y_k, full_matrices=False)
        print("Applying Soft-threshold")
        S_shrink = soft_threshold(S, (1 / mu_k))
        L_k = np.dot(U * S_shrink, V)

        # Resolver S_{k+1} = argmin_S L(L_k, S, Y_k, mu_k)
        G_S = D - L_k + (1 / mu_k) * Y_k
        
        S_k = spams.proximalGraph(np.asfortranarray(G_S), 
                                  tree, 
                                  False, 
                                  lambda1=lamb, 
                                  regul='graph')
        
        print("Updating Y_k")
        # Paso en valor del dual
        Y_k += mu_k * (D - L_k - S_k)
        print(augmented_lagrangian(L=L_k, S=S_k, lamb=lamb, mu=mu_k, Y=Y_k, D=D))

        err.append(np.linalg.norm(D - L_k - S_k, ord='fro') / np.linalg.norm(D, ord='fro'))
        if err[-1] < 1e-10:
            print('Method converged in iteration {}'.format(i))
            break


    L = L_k.copy()
    S = S_k.copy()
    
    return L, S, err

In [ ]:
import time

D, video_size = load_video("test.avi")

start = time.time()
print('Started video segmentation')

L, S, err = video_segmentation(D, video_size, max_iter=100, mu_0=.001, lamb=0.001)

print('Finished video segmentation')
print(time.time() - start)

Started video segmentation
Getting groups
Building tree
Iteracion 1
Starting SVD
Applying Soft-threshold
Updating Y_k
98854.15467425127
Iteracion 2
Starting SVD
Applying Soft-threshold
Updating Y_k
88764.97168062534
Iteracion 3
Starting SVD
Applying Soft-threshold
Updating Y_k
84100.30254233468
Iteracion 4
Starting SVD
Applying Soft-threshold
Updating Y_k
80837.77384921983
Iteracion 5
Starting SVD


In [ ]:
plt.loglog(err)
plt.title('Error relativo en norma Frobenius (Liu)')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.subplot('221')
plt.title('Low rank')
plt.imshow(L[:, 0].reshape(video_size), cmap='gray')

plt.subplot('222')
plt.title('Sparse')
plt.imshow(S[:, 0].reshape(video_size), cmap='gray')

plt.subplot('223')
plt.title('Low rank')
plt.imshow(L[:, 50].reshape(video_size), cmap='gray')

plt.subplot('224')
plt.title('Sparse')
plt.imshow(S[:, 50].reshape(video_size), cmap='gray')

In [ ]:
import cv2
import numpy as np

save_video('test_output.avi', video_size, D, L, S)